# Geometrias das faixas de pedestre

In [2]:
## TODO

# Encontrar a linha de percurso da faixa de pedestre
# Estender o polígono até as calçadas
# Salvar os polígonos

In [14]:
import geopandas as gpd
import rasterio
import numpy as np
from sklearn.cluster import DBSCAN
from shapely.geometry import MultiPoint, LineString
from shapely.ops import substring, split, nearest_points
from scipy.spatial import distance
import math
import os
from rasterio.enums import Resampling
import glob
import re

In [4]:
url = 'https://github.com/AndaSampa/poligono-de-vias/raw/master/resultado/90_poligono_de_vias_de_vila_mariana.gpkg'
gdf_pvias = gpd.read_file(url)
gdf_pvias_buffered = gdf_pvias.buffer(-1).unary_union

In [5]:
url_rotas_mediais = 'https://github.com/AndaSampa/rotas-em-calcadas/raw/main/resultados/rotas-mediais-vila-mariana.gpkg'
gdf_rotas_mediais = gpd.read_file(url_rotas_mediais)

In [6]:
rasters = glob.glob('resultados/raster/faixas-*.jp2')

In [7]:
faixas_list = []
epilson = 6

In [8]:
def quadri_metrics(coords):
    
    distances, angles = [], []

    for i, v in enumerate(coords):
        if i == 0 : continue
        distances.append(distance.euclidean(coords[i-1], v))
        angle_dif = np.array(coords[i-1]) - np.array(v)
        angles.append(math.atan2(angle_dif[0], angle_dif[1]))

    distances = np.array(distances)

    return np.min(distances), np.max(distances), angles[distances.argmin()], angles[distances.argmax()]

    

In [9]:
for f in rasters:
    dataset = rasterio.open(f)
    scm = re.findall(r'[0-9]+-[0-9]+', f)[0]
    print(f'Processando: SCM {scm}')
    data = dataset.read()
    faixas_ij = np.array([np.where(data == 1)[2], np.where(data == 1)[1]]).transpose()
    t = dataset.transform
    faixas_xy = faixas_ij * (t[0], t[4]) + (t[2], t[5]) + (t[0]/2, t[4]/2)
    faixas_list.append(faixas_xy)

    geometries = gpd.points_from_xy(x=faixas_xy[:, 0], y=faixas_xy[:, 1], crs='EPSG:31983')
    gdf_faixas = gpd.GeoDataFrame(geometry=geometries)
    
    gdf_faixas = gpd.sjoin(gdf_faixas, gdf_pvias, how='left', predicate='within')
    gdf_faixas.loc[gdf_faixas.index_right.isna(), ['index_right']] = 0
    gdf_faixas.loc[:, ['index_right']] = gdf_faixas.loc[:, ['index_right']] * (epilson + 1) + epilson + 1

    coords = np.array([gdf_faixas.geometry.x, gdf_faixas.geometry.y, gdf_faixas.index_right]).transpose()

    clusters = DBSCAN(eps=epilson).fit_predict(coords)
    
    gdf_faixas.loc[:, ['cluster']] = clusters

    # gdf_faixas.to_file(f'resultados/vetor/faixas-{scm}.gpkg', driver='GPKG')

    faixas_poly = gdf_faixas.groupby('cluster')['geometry'].apply(lambda x: MultiPoint(list(x)).minimum_rotated_rectangle)
    
    # Remove polígonos com área menor que 2m
    faixas_poly = faixas_poly[faixas_poly.geometry.area > 2]

    # Remove os polígonos que o ponto de inacessibilidade esteja 
    # fora de um buffer negativo de 1m do polígono de via
    faixas_poly = faixas_poly[faixas_poly.geometry.apply(lambda x: x.representative_point()).intersects(gdf_pvias_buffered)]

    # faixas_poly[:, ['largura', 'extensão', 'angulo da largura', 'angulo da extensão']] = \
    #     quadri_metrics()

    faixas_poly = gpd.GeoDataFrame(faixas_poly.geometry.exterior.apply(lambda x: quadri_metrics(x.coords)).to_list(), 
                                    index=faixas_poly.index, 
                                    geometry=faixas_poly.geometry, 
                                    columns=['largura', 'extensão', 'angulo da largura', 'angulo da extensão'])

    # TODO
    # Desenhar a rota de travessia
    # 
    # 1. Encontrar a rota medial mais próxima
    medial_proxima = faixas_poly.sjoin_nearest(gdf_rotas_mediais, how='left', max_distance=8.0, distance_col='distancia')

    # 2. Obter os pontos entre a rota medial e o polígono
    
    # 3. Obter o ponto invertido dentro do polígono
    # gdf_pvias.geometry.apply(lambda x: substring(x.exterior, start_dist=0.0, end_dist=0.10, normalized=True))

    # 4. Conectar o ponto invertido do polígono às rotas mediais unificadas
    # 5. Desenhar a linha entre todos os pontos
    # 6. Gerar um offset com a largura da faixa a partir da linha de travessia
    # 7. Cortar o novo polígono para fficar contido no polígono de vias                                    
    
    faixas_poly.to_file(f'resultados/vetor/faixas-poly-{scm}.gpkg', driver='GPKG')
    # gdf_faixas.plot()

    break
    

Processando: SCM 3316-214


In [35]:
# medial_proxima.loc[:, ['medial2']] = gdf_rotas_mediais.iloc[medial_proxima.index_right].geometry
medial_proxima.loc[:, :]

,largura,extensão,angulo da largura,angulo da extensão,geometry,index_right,distancia,medial,medial2
cluster,,,,,,,,,
0,4.200000,11.400000,3.141593,1.570796,"POLYGON ((331827.003 7390358.727, 331827.003 7...",319,2.412017,NaN,NaN
2,4.200000,5.640000,1.570796,3.141593,"POLYGON ((331834.203 7390355.847, 331834.203 7...",319,1.226548,NaN,NaN
4,5.291840,18.604645,-0.982794,0.588003,"POLYGON ((331829.726 7390346.552, 331819.406 7...",411,0.952454,NaN,NaN
5,5.473894,12.074767,-1.107149,0.463648,"POLYGON ((331870.635 7390278.351, 331865.235 7...",329,1.323164,NaN,NaN
8,6.043053,11.266946,0.200653,1.771449,"POLYGON ((331879.976 7390263.619, 331868.935 7...",409,2.698761,NaN,NaN
9,2.760000,4.200000,3.141593,1.570796,"POLYGON ((331906.203 7390256.487, 331906.203 7...",403,3.640586,NaN,NaN
10,2.760000,2.760000,3.141593,1.570796,"POLYGON ((331907.643 7390249.287, 331907.643 7...",403,2.137244,NaN,NaN
11,5.640000,11.400000,3.141593,1.570796,"POLYGON ((331995.483 7390213.287, 331995.483 7...",403,0.928258,NaN,NaN
13,4.836610,16.800857,-2.356194,2.356194,"POLYGON ((332039.763 7390202.247, 332051.643 7...",413,0.000000,NaN,NaN


In [41]:
medial_proxima.apply(lambda x: nearest_points(x.geometry, 
                                            gdf_rotas_mediais.iloc[x.index_right].geometry), axis=1)

cluster
0     (POINT (331827.00301 7390362.926980002), POINT...
2     (POINT (331830.9887517345 7390361.486980001), ...
4     (POINT (331823.8091638461 7390328.136210771), ...
5     (POINT (331875.53101 7390275.902980002), POINT...
8     (POINT (331878.7719037931 7390257.697374484), ...
9     (POINT (331906.20301 7390256.486980001), POINT...
10    (POINT (331907.64301 7390252.046980002), POINT...
11    (POINT (331984.08301 7390215.424628947), POINT...
13    (POINT (332044.2862629271 7390204.563727071), ...
15    (POINT (332017.2657504089 7390195.600904903), ...
17    (POINT (332053.2460421491 7390177.243947855), ...
26    (POINT (331960.5517314696 7390006.614168718), ...
46    (POINT (331618.20301 7389866.246980002), POINT...
48    (POINT (331982.64301 7389861.806980002), POINT...
dtype: object

In [19]:
medial_proxima.geometry

cluster
0     POLYGON ((331827.003 7390358.727, 331827.003 7...
2     POLYGON ((331834.203 7390355.847, 331834.203 7...
4     POLYGON ((331829.726 7390346.552, 331819.406 7...
5     POLYGON ((331870.635 7390278.351, 331865.235 7...
8     POLYGON ((331879.976 7390263.619, 331868.935 7...
9     POLYGON ((331906.203 7390256.487, 331906.203 7...
10    POLYGON ((331907.643 7390249.287, 331907.643 7...
11    POLYGON ((331995.483 7390213.287, 331995.483 7...
13    POLYGON ((332039.763 7390202.247, 332051.643 7...
15    POLYGON ((332016.546 7390193.375, 332026.124 7...
17    POLYGON ((332055.243 7390175.247, 332060.103 7...
26    POLYGON ((331976.952 7390000.358, 331960.552 7...
46    POLYGON ((331618.203 7389866.247, 331618.203 7...
48    POLYGON ((331986.843 7389856.167, 331986.843 7...
Name: geometry, dtype: geometry